In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from cosy.models import CosyNet
from cosy.losses import trace_norm, MaskedMeanSquaredError, mutual_information, l1_norm, wasserstein_distance

In [2]:
df = pd.read_csv("/Users/tompope/Documents/python_enviroment/SoftwareDevelopment/soft_parameter_sharing/data/desulferization_data_w_tobacc.csv")
df.head()

,Unnamed: 0,framework,CO2_tp_lod,N2_tp_lod,SO2_tp_lod,RDF_electronegativity_2.00,RDF_electronegativity_2.00.1,RDF_electronegativity_2.01,RDF_electronegativity_2.03,RDF_electronegativity_2.04,...,RDF_vdWaalsVolume_28.53,RDF_vdWaalsVolume_29.01,RDF_vdWaalsVolume_29.51,RDF_vdWaalsVolume_30.00,PLD,LCD,density,VF,SAv,SAg
0,0,GIQXUA_clean,1.918899,0.017520,2.747258,0.000605,0.000606,0.000607,0.000609,0.000614,...,0.0,0.0,0.0,0.0,3.86870,4.89334,1.792830,0.13844,2675.750,1492.470
1,1,JALPAQ_clean,0.050226,0.027788,0.008087,0.000861,0.000870,0.000888,0.000916,0.000954,...,0.0,0.0,0.0,0.0,2.90553,3.97550,1.660880,0.01992,841.869,506.881
2,2,SAQQEH_clean,2.006302,0.249367,0.533491,0.000771,0.000781,0.000802,0.000833,0.000876,...,0.0,0.0,0.0,0.0,9.13279,10.56525,0.914389,0.35942,2030.210,2220.290
3,3,BEFNEH_clean,1.036156,0.033717,1.343768,0.000899,0.000899,0.000900,0.000903,0.000910,...,0.0,0.0,0.0,0.0,3.33494,4.26940,2.165640,0.07290,1864.990,861.172
4,4,GELVID_clean,0.552087,0.064768,0.027879,0.000820,0.000831,0.000852,0.000884,0.000928,...,0.0,0.0,0.0,0.0,2.43534,3.61901,1.574130,0.02776,1334.150,847.547


In [3]:
'''
  - read data into dataframe
  - generate class data
  - extract and split feature and class data
  - scale feature data
'''


df = df.sample(frac=1).reset_index(drop=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
# df = df[df.framework.str.startswith('str_')]


train_df_full, test_df = train_test_split(df, test_size=0.2, random_state=2020)

train_df, val_df = train_test_split(train_df_full, test_size=0.1, random_state=2020)

X_train = train_df.iloc[:, 4:].to_numpy().astype(np.float32)
X_train_CO2 = train_df.iloc[:, 4:].to_numpy().astype(np.float32)

y_CO2 = train_df.CO2_tp_lod.to_numpy().astype(np.float32)
y_N2 = train_df.N2_tp_lod.to_numpy().astype(np.float32)
y_SO2 = train_df.SO2_tp_lod.to_numpy().astype(np.float32)

X_val = val_df.iloc[:, 4:].to_numpy().astype(np.float32)
X_val_CO2 = val_df.iloc[:, 4:].to_numpy().astype(np.float32)
y_CO2_val = val_df.CO2_tp_lod.to_numpy().astype(np.float32)
y_N2_val = val_df.N2_tp_lod.to_numpy().astype(np.float32)
y_SO2_val = val_df.SO2_tp_lod.to_numpy().astype(np.float32)

X_test = test_df.iloc[:, 4:].to_numpy().astype(np.float32)
X_test_CO2 = test_df.iloc[:, 4:].to_numpy().astype(np.float32)
y_CO2_test =  test_df.CO2_tp_lod.to_numpy().astype(np.float32)
y_N2_test = test_df.N2_tp_lod.to_numpy().astype(np.float32)
y_SO2_test = test_df.SO2_tp_lod.to_numpy().astype(np.float32)


# y_CO2[100:1000] = -1

sc = StandardScaler()


X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [4]:
df.head()

,framework,CO2_tp_lod,N2_tp_lod,SO2_tp_lod,RDF_electronegativity_2.00,RDF_electronegativity_2.00.1,RDF_electronegativity_2.01,RDF_electronegativity_2.03,RDF_electronegativity_2.04,RDF_electronegativity_2.07,...,RDF_vdWaalsVolume_28.53,RDF_vdWaalsVolume_29.01,RDF_vdWaalsVolume_29.51,RDF_vdWaalsVolume_30.00,PLD,LCD,density,VF,SAv,SAg
0,VIGNOQ_clean,0.227582,0.083390,0.258562,0.000897,0.000907,0.000926,0.000955,0.000993,0.001039,...,0.0,0.0,0.0,0.0,3.12932,8.72015,1.429790,0.20066,2099.680,1468.520
1,str_m9_o4_o4_f0_sra_sym_60_faps_mpnn_charges,0.629119,0.096333,0.191775,0.000928,0.000940,0.000963,0.000999,0.001047,0.001109,...,0.0,0.0,0.0,0.0,4.59862,5.79147,1.440720,0.06678,969.055,672.617
2,str_m2_o23_o23_f0_nbo_sym_89_faps_mpnn_charges,0.155770,0.284778,0.031045,0.000788,0.000798,0.000817,0.000847,0.000888,0.000942,...,0.0,0.0,0.0,0.0,16.11962,25.23128,0.204334,0.77962,1230.860,6023.770
3,str_m3_o56_o13_f0_fsc_sym_27_faps_mpnn_charges,1.552785,0.296811,1.096783,0.000817,0.000833,0.000866,0.000917,0.000989,0.001083,...,0.0,0.0,0.0,0.0,4.97325,7.58466,1.025480,0.23276,2526.730,2463.940
4,str_m29_o80_o45_f0_pts_sym_17_faps_mpnn_charges,1.241040,0.240859,1.932541,0.000765,0.000778,0.000806,0.000848,0.000906,0.000980,...,0.0,0.0,0.0,0.0,7.41119,9.12364,0.694914,0.35748,2679.460,3855.810


In [5]:
input_ = tf.keras.Input(shape=(X_train.shape[1]))
x = tf.keras.layers.Dense(1200, activation='relu')(input_)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(400, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.Model(inputs=input_, outputs=x)

Metal device set to: Apple M1


2023-02-21 16:17:19.685421: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-21 16:17:19.685878: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
input_ = tf.keras.Input(shape=(X_train_CO2.shape[1]))
x = tf.keras.layers.Dense(1200, activation='relu')(input_)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(400, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

modelco2 = tf.keras.Model(inputs=input_, outputs=x)

In [7]:
cosy_model = CosyNet(
    model_config=[model.get_config(), modelco2.get_config(), model.get_config()],
    number_models=3,
    scalar=[1.0, 1.0, 1.0],
    max_layer_cutoff=-1,
    min_layer_cutoff=1,
    loss_fn=wasserstein_distance,
)

In [8]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00025)

losses = {
    'output_1':MaskedMeanSquaredError(),
    'output_2': MaskedMeanSquaredError(),
    'output_3': MaskedMeanSquaredError(),
    }

R2 = tfa.metrics.RSquare()

cosy_model.compile(
            optimizer=opt,
            loss=losses,
            metrics=[R2],
            )

In [9]:
cosy_model.fit(
    x=X_train, y=[y_N2, y_CO2, y_SO2],
    validation_data=(X_val, [y_N2_val, y_CO2_val, y_SO2_val]),
    batch_size=32, epochs = 4, verbose=1
)

Epoch 1/4


2023-02-21 16:17:20.026393: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-21 16:17:20.197799: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 16:17:20.200010: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 16:17:20.230485: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 16:17:20.232882: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 16:17:20.266577: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 16:17:20.270745: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INT

274/878 [========>.....................] - ETA: 1:19 - loss: 2.3255 - output_1_loss: 0.0237 - output_2_loss: 0.3001 - output_3_loss: 0.8224 - output_1_r_square: -2.2002 - output_2_r_square: 0.1807 - output_3_r_square: 0.3704 - scaled_soft_loss: 0.3931

KeyboardInterrupt: 

In [ ]:
models = cosy_model.get_models()

In [ ]:
params = cosy_model._get_parameters()

In [ ]:
from cosy.losses import trace_norm, squared_frobenius_norm

In [ ]:
trace_norm(params, [0.5, 0.5, 0.5])

In [ ]:
squared_frobenius_norm(params, [0.5, 0.5, 0.5])

In [ ]:
from sklearn.metrics import r2_score
cosy_model.evaluate(X_test, [y_N2_test, y_CO2_test, y_SO2_test])
predictions = cosy_model.predict(X_test)

R2 = tfa.metrics.RSquare()

pred1 = np.array(predictions[0]).reshape(-1)
pred2 = np.array(predictions[1]).reshape(-1)
pred3 = np.array(predictions[2]).reshape(-1)


print(r2_score(y_N2_test, pred1))
print(r2_score(y_CO2_test, pred2))
print(r2_score(y_SO2_test, pred3))

In [ ]:
import itertools


def squared_frobenius_norm(parameters, lambdas):
    """Computes the overall loss for soft parameter sharing between multiple neural networks."""
    losses = []
    for W, lambd in zip(parameters, lambdas):
        losses.append(pairwise_loss_squared_frobenius(W, lambd))
    return tf.reduce_sum(losses)

In [ ]:
import numpy as np
parameters = np.array([
        [tf.zeros((2, 2)), tf.ones((2, 2)), tf.zeros((2, 2))],
        [tf.ones((2, 2)), tf.ones((2, 2)), tf.ones((2, 2))],
        [tf.ones((2, 2)), tf.zeros((2, 2)), tf.ones((2, 2))],
    ])

lambdas = [0.5, 0.5, 0.5]

In [ ]:
loss = 0
for lam, param in zip(lambdas, parameters):
    norm1 = np.linalg.norm(param[0] - param[1], ord='fro') ** 2
    norm2 = np.linalg.norm(param[0] - param[2], ord='fro') ** 2
    norm3 = np.linalg.norm(param[1] - param[2], ord='fro') ** 2
    loss += lam * (norm1 + norm2 + norm3)

In [ ]:
loss

In [ ]:
loss = 0
for lam, param in zip(lambdas, parameters):
    norm1 = np.linalg.norm(param[0] - param[1], ord='nuc') ** 2
    norm2 = np.linalg.norm(param[0] - param[2], ord='nuc') ** 2
    norm3 = np.linalg.norm(param[1] - param[2], ord='nuc') ** 2
    loss += lam * (norm1 + norm2 + norm3)

In [ ]:
loss

In [ ]:
weights = [np.array([[1, 2], [3, 4]]), np.array([[2, 4], [6, 8]])]

In [ ]:
def pairwise_loss_squared_frobenius(W, lambd):
    """Computes the pairwise weight-sharing loss between a list of weight matrices."""
    squared_norms = []
    for Wi, Wj in itertools.combinations(W, 2):
        squared_norms.append(tf.norm(Wi - Wj, ord="fro", axis=(0, 1)) ** 2)
    loss = lambd * tf.reduce_sum(squared_norms)
    return loss


In [ ]:
norm1 = np.linalg.norm(weights[0] - weights[1], ord='fro') ** 2

In [ ]:
norm1

In [ ]:
W_tf = [tf.constant(Wi, dtype=tf.float32) for Wi in weights]
pairwise_loss_squared_frobenius(W_tf, 1.0)